**HindcastFile**: File Type - contains link back up to *HindcastArchive* and has a link to a *C3.File* type. 

In [1]:
save_file1 = c3.HindcastFile.get('007e2816-3905-434b-b957-f7b0d0b03f46')
save_file2 = c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb')

In [2]:
save_file2

c3.HindcastFile(
 id='007e88f7-b86d-4379-a194-a73e2d003afb',
 name='GOMu0.04-expt_90.1m000-2021-2021-09-23T23:00:00Z.nc',
 meta=c3.Meta(
        tenantTagId=150,
        tenant='dev',
        tag='tc01d',
        created=datetime.datetime(2021, 10, 15, 16, 9, 43, tzinfo=datetime.timezone.utc),
        createdBy='dadams@illinois.edu',
        updated=datetime.datetime(2021, 10, 15, 16, 13, 20, tzinfo=datetime.timezone.utc),
        updatedBy='worker',
        timestamp=datetime.datetime(2021, 10, 15, 16, 13, 20, tzinfo=datetime.timezone.utc),
        fetchInclude='[]',
        fetchType='HindcastFile'),
 version=3,
 hindcastArchive=c3.HindcastArchive(id='7a187ef2-d88d-4658-b19f-451aef87535f'),
 subsetOptions=c3.HycomSubsetOptions(
                 timeRange=c3.TimeRange(
                             start=datetime.datetime(2021, 9, 23, 23, 0),
                             end=datetime.datetime(2021, 9, 23, 23, 0)),
                 timeStride=1,
                 vars='water_u,water_v',


In [3]:
# hycom_file = c3.HycomUtil.nc_open(save_file.file.url)
# # save_file.file

In [4]:
# from datetime import datetime, timedelta
# # hycom_file['time']

# time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
#                                         '%Y-%m-%d %H:%M:%S %z')

In [4]:
# process(c3.HindcastFile.get('007e2816-3905-434b-b957-f7b0d0b03f46'))
# process(c3.HindcastFile.get('007e88f7-b86d-4379-a194-a73e2d003afb'))

In [5]:
#

In [11]:
# Prototype for processing hindcast files
from datetime import datetime, timedelta
def process(this):
    """ Process a single Hindcast NetCDF file into the Hindcast__Data types"""
    # extract surface data for a variable
    hycom_file = c3.HycomUtil.nc_open(this.file.url)
    # One time stamp
    time_origin = datetime.strptime(hycom_file.variables['time'].__dict__['time_origin'] + ' +0000',
                                        '%Y-%m-%d %H:%M:%S %z')
    
    # extract lat-long, or derive this from types
    # Not yet done:determine the offset for each based on the subsetOptions for this file
    # Note: for now it's just an integer list
    # 159 latitudes, 276 longitudes for one time stamp
    latitudes = range(len(hycom_file['lat']))
    longitudes = range(len(hycom_file['lon']))
    time = time_origin + timedelta(hours=hycom_file.variables['time'][:][0])

    # Create list of instantiated SurfaceHindcastData types
    # Create a parent id for each lat-long pair in the file.
    # use lat-long indicies to create a string parent id

    data_records = [
        # Lambda function! 
        # Creates SurfaceHindCastData entry for each i and j in latitudes and longitudes  
        c3.SurfaceHindcastData( # look at the code to understand, TimeDataPoint (Check out Type)
            **{
                'start': time,
                'parent': 'HNDCST_SRFC_' + str(i) + '-' + str(j),
                'name': 'water_u',  # variable
                'for': time,           # timestamp
                'water_u': hycom_file.variables['water_u'][:].data[0,0,i,j],
                'water_v': hycom_file.variables['water_v'][:].data[0,0,i,j]
            }
        )
        for i in latitudes[:1]
            for j in longitudes[:1]
    ]
    # upsert to data store
    # upload this!
    c3.SurfaceHindcastData.upsertBatch(data_records)

    # close the file ds, url
    c3.HycomUtil.nc_close(ds=hycom_file, url=this.file.url)
    
    # aggregation in space is possible but a bit more complex

In [12]:
process(save_file1)

In [ ]:
data_records = [
        c3.SurfaceHindcastDataSeries(
            **{ 'id': 'HNDCST_SRFC_0-0',
                'latLongPair': 'GOMu0.04_1-1',
                
            }
        )
    ]

In [7]:
c3.SurfaceHindcastDataSeries.upsertBatch(data_records)

c3.ObjList<SurfaceHindcastDataSeries>(
 stats=c3.Stats(
         createdObjCount=1,
         updatedObjCount=0,
         removedObjCount=0,
         failedObjCount=0,
         objCount=1,
         dbWait=0.0,
         time=0.181,
         dbTime=0.013696359,
         objsPerSecond=5.524861878453039,
         dbObjsPerSecond=73.012104895907),
 objs=c3.Arry<SurfaceHindcastDataSeries>([c3.SurfaceHindcastDataSeries(
         id='HNDCST_SRFC_1-1',
         meta=c3.Meta(
                created=datetime.datetime(2021, 10, 15, 23, 48, 19, tzinfo=datetime.timezone.utc),
                updated=datetime.datetime(2021, 10, 15, 23, 48, 19, tzinfo=datetime.timezone.utc),
                timestamp=datetime.datetime(2021, 10, 15, 23, 48, 19, tzinfo=datetime.timezone.utc)),
         version=1)]))

In [8]:
# Define simple metric
my_metric = c3.SimpleMetric(id = "AverageWaterU_HycomLatLongPair",
                            name = "AverageWaterU",
                            description = "Calculates average surface X-velocity for the given interval",
                            srcType = "HycomLatLongPair",
                            path = "surfaceHindcastData",
                            expression = "avg(avg(data.water_u))"
                           )

my_metric.toJson()
# Create EvalMetricsSpec
my_spec = c3.EvalMetricsSpec(
            ids = ["GOMu0.04_0-0"], #, "GOMu0.04_1-1"],
            expressions = ["AverageWaterU"], # does this refer to the one above?
            start = "2021-09-01",
            end = "2021-09-02",
            interval = "DAY"
        )

In [9]:
evalMetricsResult = c3.HycomLatLongPair.evalMetricsWithMetadata(spec=my_spec,
                                                      overrideMetrics=[my_metric])
evalMetricsResult

c3.EvalMetricsResult(
 result=c3.Mapp<string, map<string,Timeseries>>({'GOMu0.04_0-0': c3.Mapp<string, Timeseries>({'AverageWaterU': c3.NormTimeseriesDouble(
                                             m_start=datetime.datetime(2021, 9, 1, 0, 0),
                                             m_end=datetime.datetime(2021, 9, 2, 0, 0),
                                             m_data=c3.Arry<double>([-0.13050000928342342]),
                                             m_interval='HOUR',
                                             m_timeZone='NONE',
                                             m_sourceInfo=c3.TimeseriesSourceInfo(
                                                            source=c3.Ref(
                                                                     moduleName='-type-HycomLatLongPair',
                                                                     typeName='HycomLatLongPair',
                                                                     id='GOMu0.04

In [12]:
# convert EvalMetricsResult to Pandas DataFrame
df = c3.EvalMetricsResult.toPandas(result=evalMetricsResult)
df

,AverageWaterU
GOMu0.04_0-0_2021-09-01T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-02T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-03T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-04T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-05T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-06T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-07T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-08T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-09T00:00:00,-0.1305
GOMu0.04_0-0_2021-09-10T00:00:00,-0.1305


In [ ]:
# Next Steps:
# 1) Understand the math functions and why it does what it does?
# 2) How can we do lat-lon filtering?
# 3) Implement full pipeline for a specific use-case and compare time-performance